In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
left_image = cv2.imread(os.path.join(DATA_FOLDER, "keparoi_left_frame_aligned.jpg"))
right_image = cv2.imread(os.path.join(DATA_FOLDER, "keparoi_right_frame_aligned.jpg"))

camera_distortion = np.load(os.path.join(DATA_FOLDER, "camera_distortion.npy"))
intrinsic_matrix = np.load(os.path.join(DATA_FOLDER, "camera_matrix.npy"))
undistorted_camera_matrix = np.load(os.path.join(DATA_FOLDER, "undistorted_camera_matrix.npy"))

In [ ]:
print(camera_distortion)
print(intrinsic_matrix)

In [ ]:
left_image_padded = cv2.copyMakeBorder(
     left_image, 
     200, 
     200, 
     200, 
     200, 
     cv2.BORDER_CONSTANT, 
     value=[0, 0, 0]
)

right_image_padded = cv2.copyMakeBorder(
     right_image, 
     200, 
     200, 
     200, 
     200, 
     cv2.BORDER_CONSTANT, 
     value=[0, 0, 0]
)

print(left_image_padded.shape)
plt.imshow(left_image_padded)
plt.show()

In [ ]:
# CAMERA PARAMS

ppx = 365
ppy = 365
aspect = 1
focal = 2442
# focal 2425

K = np.array([[1.8908248e+03, 0.0000000e+00, 3.6500000e+02],
 [0.0000000e+00, 1.8908248e+03, 2.0550000e+02],
 [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]])

print(K)

distortion = np.array([])

In [ ]:
intrinsic_matrix = np.array([[1.87758667e+03, 0.00000000e+0, 3.65000000e+02],
 [0.00000000e+00, 1.87758667e+03, 2.05500000e+02],
 [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [ ]:
intrinsic_matrix = np.array([
    [2.7, 0, 0.145],
    [0, 2.7, 0],
    [0, 0, 0]
])

camera_distortion = np.array([[4.56e-04, 2.70e-07, 6.00e-05, 3.46e-06, 3.86e-11]])
w, h = 1920, 1080
undistorted_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(intrinsic_matrix, camera_distortion, (w,h), 1, (w,h))
print(undistorted_camera_matrix)

In [ ]:
undistorted_left_image = cv2.undistort(left_image_padded, intrinsic_matrix, camera_distortion, None, undistorted_camera_matrix)
undistorted_right_image = cv2.undistort(right_image_padded, intrinsic_matrix, camera_distortion, None, undistorted_camera_matrix)

print(undistorted_left_image.shape)

plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(left_image)
plt.subplot(1,2,2)
plt.imshow(undistorted_left_image)
plt.show()

In [ ]:
left_image_path = os.path.join(DATA_FOLDER, "left_image_undistorted.jpg")
right_image_path = os.path.join(DATA_FOLDER, "right_image_undistorted.jpg")

cv2.imwrite(left_image_path, undistorted_left_image) 
cv2.imwrite(right_image_path, undistorted_right_image)

In [ ]:
R = np.array([[1,0,0],[0,1,0],[0,0,1]])
map1, map2 = cv2.initUndistortRectifyMap(intrinsic_matrix, camera_distortion, R, undistorted_camera_matrix, 
                                         (1920, 1080), cv2.CV_16SC2)

# remap
left_image_undistorted = cv2.remap(left_image_padded, map1, map2, cv2.INTER_LINEAR)
plt.imshow(left_image_undistorted)
plt.show()